# Project Introduction: Text2SQL via Prompt Engineering

*   Adding dataset for the employees


**About Text2SQL:** Text-to-SQL, also known as Natural Language to SQL (NL2SQL), is a rapidly evolving technology that translates natural, everyday language into Structured Query Language (SQL) commands. This innovative approach empowers users to interact with and retrieve data from databases simply by asking questions in plain English, eliminating the need for specialized knowledge of complex SQL syntax.

At its core, Text-to-SQL acts as an intelligent translator. It leverages the power of artificial intelligence, particularly Natural Language Processing (NLP) and sophisticated AI models, to understand the user's intent and generate the corresponding SQL query. This process allows individuals without a technical background to explore and analyze data, thereby democratizing data access within an organization.

## Retrieve data


In [1]:
! curl "https://api.mockaroo.com/api/dde01370?count=1000&key=11149690" > "customers.csv"
! curl "https://api.mockaroo.com/api/8ba6f630?count=1000&key=11149690" > "products.csv"
! curl "https://api.mockaroo.com/api/8ba6f630?count=1000&key=11149690" > "orders.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 99620    0 99620    0     0  58386      0 --:--:--  0:00:01 --:--:-- 58359
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  684k    0  684k    0     0   259k      0 --:--:--  0:00:02 --:--:--  259k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  691k    0  691k    0     0   207k      0 --:--:--  0:00:03 --:--:--  207k


Adding employees dataset

In [5]:
! curl "https://api.mockaroo.com/api/503e3680?count=1000&key=24647200" > "employees.csv"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 92216    0 92216    0     0  57821      0 --:--:--  0:00:01 --:--:-- 57815


Loading dataset

In [6]:
import pandas as pd

try:
    employees_df = pd.read_csv('employees.csv')
    print("First 5 rows of the downloaded employees data:")
    display(employees_df.head())
except FileNotFoundError:
    print("Error: employees.csv not found. Please ensure the curl command in the previous cell ran successfully with the correct Mockaroo URL and API key.")
except Exception as e:
    print(f"An error occurred while reading employees.csv: {e}")

First 5 rows of the downloaded employees data:


,employee_id,first_name,last_name,email,gender,job_title,department,salary,hire_date
0,1,Blinni,Salaman,bsalaman0@biblegateway.com,Female,Environmental Tech,HR,76600.20,2/25/2018
1,2,Ainslee,Balcers,abalcers1@cpanel.net,Female,Research Assistant IV,Marketing,33484.14,9/2/2013
2,3,Thomasine,Liebrecht,tliebrecht2@tamu.edu,Bigender,Senior Developer,Marketing,36691.01,7/25/2017
3,4,Erasmus,Long,elong3@yale.edu,Male,Systems Administrator II,Marketing,45727.19,3/16/2012
4,5,Farrell,Timmons,ftimmons4@ibm.com,Male,Structural Engineer,HR,75516.15,9/29/2010


Setup database

In [7]:
import sqlite3
import pandas as pd
import os

In [8]:
# Define SQL schemas for creating tables
customers_schema = """
CREATE TABLE IF NOT EXISTS customers (
    customer_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(50),
    phone_number VARCHAR(50),
    address VARCHAR(50),
    city VARCHAR(50),
    country VARCHAR(50),
    postal_code VARCHAR(50),
    loyalty_points INT
);
"""

products_schema = """
CREATE TABLE IF NOT EXISTS products (
    product_id INT PRIMARY KEY,
    product_name TEXT,
    description TEXT,
    price DECIMAL(10,2),
    discount_percentage DECIMAL(5,2),
    category VARCHAR(50),
    brand TEXT,
    stock_quantity INT,
    color VARCHAR(50),
    size VARCHAR(20),
    weight DECIMAL(5,2),
    dimensions TEXT,
    release_date DATE,
    rating DECIMAL(3,1),
    reviews_count INT,
    seller_name TEXT,
    seller_rating DECIMAL(3,1),
    seller_reviews_count INT,
    shipping_method VARCHAR(20),
    shipping_cost DECIMAL(6,2)
);
"""

orders_schema = """
CREATE TABLE IF NOT EXISTS orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    product_id INT,
    quantity INT,
    unit_price DECIMAL(10,2),
    total_price DECIMAL(10,2),
    order_date DATE,
    shipping_address VARCHAR(255),
    payment_method VARCHAR(20),
    status VARCHAR(20),
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);
"""
employees_schema = """
CREATE TABLE IF NOT EXISTS employees (
    employee_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(100),
    gender VARCHAR(10),
    department VARCHAR(50),
    job_title VARCHAR(100),
    salary DECIMAL(10,2),
    hire_date DATE
);
"""

In [9]:
db_name = 'ecommerce.db'
if os.path.exists(db_name):
    os.remove(db_name)
    print(f"Removed existing database '{db_name}'.")

In [10]:
import sqlite3
import pandas as pd
import os



COLUMN_DATA_TYPES = {
    'customers': {
        'customer_id': 'int64',
        'first_name': 'object',
        'last_name': 'object',
        'email': 'object',
        'phone_number': 'object',
        'address': 'object',
        'city': 'object',
        'country': 'object',
        'postal_code': 'object',
        'loyalty_points': 'int64'
    },

  'employees': {
        'employee_id': 'int64',
        'first_name': 'object',
        'last_name': 'object',
        'email': 'object',
        'gender': 'object' ,
        'department': 'object',
        'job_title': 'object',
        'salary': 'float64',
        'hire_date': 'datetime64[ns]'
    },

    'products': {
        'product_id': 'int64',
        'product_name': 'object',
        'description': 'object',
        'price': 'float64',
        'discount_percentage': 'float64',
        'category': 'object',
        'brand': 'object',
        'stock_quantity': 'int64',
        'color': 'object',
        'size': 'object',
        'weight': 'float64',
        'dimensions': 'object',
        'release_date': 'datetime64[ns]',
        'rating': 'float64',
        'reviews_count': 'int64',
        'seller_name': 'object',
        'seller_rating': 'float64',
        'seller_reviews_count': 'int64',
        'shipping_method': 'object',
        'shipping_cost': 'float64'
    },
    'orders': {
        'order_id': 'int64',
        'customer_id': 'int64',
        'product_id': 'int64',
        'quantity': 'int64',
        'unit_price': 'float64',
        'total_price': 'float64',
        'order_date': 'datetime64[ns]',
        'shipping_address': 'object',
        'payment_method': 'object',
        'status': 'object'
    }
}

# --- Database setup ---
db_name = 'ecommerce.db'
conn = None  # Initialize connection to None

try:
    # Establish a connection to the SQLite database
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    print(f"Database '{db_name}' created and connected successfully. ✅")

    # Create tables
    cursor.execute(customers_schema)
    cursor.execute(products_schema)
    cursor.execute(orders_schema)
    cursor.execute(employees_schema)
    conn.commit()
    print("Tables 'customers', 'products', and 'orders' created successfully.")

    # --- Load data from CSV files into the tables using pandas ---
    csv_to_table_map = {
        '/content/customers.csv': 'customers',
        '/content/employees.csv': 'employees',
        '/content/products.csv': 'products',
        '/content/orders.csv': 'orders'
    }

    for csv_file, table_name in csv_to_table_map.items():
        if os.path.exists(csv_file):
            print(f"\nProcessing '{csv_file}' for table '{table_name}'...")

            # Read the CSV file into a pandas DataFrame
            df = pd.read_csv(csv_file)

            # 1. Get the expected schema for the current table
            expected_schema = COLUMN_DATA_TYPES[table_name]
            expected_cols = list(expected_schema.keys())

            # 2. Handle missing/extra columns
            # Drop columns from DataFrame that are not in the schema
            df = df[df.columns.intersection(expected_cols)]

            # Add any missing columns and fill with None (which becomes NULL in SQL)
            for col in expected_cols:
                if col not in df.columns:
                    df[col] = None

            # 3. Reorder columns to match the defined schema exactly
            df = df[expected_cols]

            # 4. Enforce data types
            for col, dtype in expected_schema.items():
                if 'datetime' in dtype:
                    # Use pd.to_datetime for date/time columns, coercing errors to NaT (Not a Time)
                    df[col] = pd.to_datetime(df[col], errors='coerce')
                else:
                    # Use astype for other columns, handling potential conversion errors
                    try:
                        df[col] = df[col].astype(dtype)
                    except (ValueError, TypeError) as e:
                        print(f"  - Warning: Could not convert column '{col}' to {dtype}. Error: {e}. Leaving as is.")


            # Use the to_sql method to insert the cleaned DataFrame
            df.to_sql(table_name, conn, if_exists='append', index=False)
            print(f"  -> Data from '{csv_file}' loaded into '{table_name}' table successfully.")
        else:
            print(f"Warning: '{csv_file}' not found. Skipping data load for '{table_name}'.")

    # Commit the changes to the database
    conn.commit()
    print("\nData committed to the database successfully. 🎉")

except sqlite3.Error as e:
    print(f"Database error: {e}")
except pd.errors.EmptyDataError as e:
    print(f"Pandas error: {e}. One of the CSV files might be empty.")
except KeyError as e:
    print(f"Schema definition error: A column is missing from the TABLE_DATA_TYPES dictionary: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
    # Close the connection if it was established
    if conn:
        conn.close()
        print("Database connection closed.")

Database 'ecommerce.db' created and connected successfully. ✅
Tables 'customers', 'products', and 'orders' created successfully.

Processing '/content/customers.csv' for table 'customers'...
  -> Data from '/content/customers.csv' loaded into 'customers' table successfully.

Processing '/content/employees.csv' for table 'employees'...
  -> Data from '/content/employees.csv' loaded into 'employees' table successfully.

Processing '/content/products.csv' for table 'products'...
  -> Data from '/content/products.csv' loaded into 'products' table successfully.

Processing '/content/orders.csv' for table 'orders'...
  - Warning: Could not convert column 'order_id' to int64. Error: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'. Leaving as is.
  - Warning: Could not convert column 'customer_id' to int64. Error: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'. Leaving as is.
  - Warning: Could not convert column 'qua

### Install Gen AI library

In [11]:
!pip install google-genai

In [12]:
from google import genai
from google.colab import userdata
genai_client = genai.Client(api_key=userdata.get('API_KEY'))

In [13]:
prompt = """
### **ROLE**

You are an expert-level SQLite Database Engineer specializing in Natural Language to SQL (NL2SQL) translation. Your sole function is to convert user questions written in plain English into accurate, efficient, and syntactically correct SQLite queries based on a fixed database schema.

-----

### **CONTEXT**

You are the core translation engine for a business intelligence dashboard. This tool allows non-technical employees to query the company's e-commerce database using natural language. The database dialect is always **SQLite**. Your responses will be executed directly on the database.

The database consists of the following three tables:

**`customers` table:**

```sql
CREATE TABLE customers (
    customer_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(50),
    phone_number VARCHAR(50),
    address VARCHAR(50),
    city VARCHAR(50),
    country VARCHAR(50),
    postal_code VARCHAR(50),
    loyalty_points INT
);
```
*** 'employees' table:***

```sql
CREATE TABLE employees (
    employee_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(100),
    gender VARCHAR(10),
    department VARCHAR(50),
    job_title VARCHAR(100),
    salary DECIMAL(10,2),
    hire_date DATE
);
```
**`products` table:**

```sql
CREATE TABLE products (
    product_id INT PRIMARY KEY,
    product_name TEXT,
    description TEXT,
    price DECIMAL(10,2),
    discount_percentage DECIMAL(5,2),
    category VARCHAR(50),
    brand TEXT,
    stock_quantity INT,
    color VARCHAR(50),
    size VARCHAR(20),
    weight DECIMAL(5,2),
    dimensions TEXT,
    release_date DATE,
    rating DECIMAL(3,1),
    reviews_count INT,
    seller_name TEXT,
    seller_rating DECIMAL(3,1),
    seller_reviews_count INT,
    shipping_method VARCHAR(20),
    shipping_cost DECIMAL(6,2)
);
```

**`orders` table:**

```sql
CREATE TABLE orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    product_id INT,
    quantity INT,
    unit_price DECIMAL(10,2),
    total_price DECIMAL(10,2),
    order_date DATE,
    shipping_address VARCHAR(255),
    payment_method VARCHAR(20),
    status VARCHAR(20),
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);
```

-----

### **TASK**

Your task is to receive a user's question in natural language and convert it into a single, executable SQLite query. Follow these steps meticulously:

1.  **Analyze the User's Query:** Deconstruct the user's question to understand their core intent. Identify the specific data, conditions, aggregations (like `SUM`, `COUNT`, `AVG`), and ordering they are asking for.
2.  **Map to the Schema:** Map the entities from the user's query to the appropriate tables (`customers`, `products`, `orders`) and columns. Determine the necessary `JOIN` operations using `customers.customer_id` and `products.product_id` as foreign keys in the `orders` table.
3.  **Construct the SQLite Query:** Write a clean and efficient `SELECT` statement that is syntactically correct for SQLite. Ensure all table and column names are accurate.
4.  **Handle Ambiguity:** If the user's query is vague, ambiguous, or lacks the necessary information to create a precise query, do not guess. Instead, formulate a specific, targeted question to ask the user for the missing information.

-----

### **CONSTRAINTS**

  * **Read-Only Operations:** You must **ONLY** generate `SELECT` queries. Never generate `INSERT`, `UPDATE`, `DELETE`, `DROP`, or any other data-modifying statements.
  * **Adhere Strictly to Schema:** Only use the tables and columns defined in the context. Do not invent or assume the existence of any other tables or columns.
  * **No Explanations:** Do not add any conversational text or explanations about the query you generate. Your output must strictly follow the specified format.
  * **Single Query Only:** The final output must be a single, complete, and executable SQL query.
  * **Handle Impossibility:** If a request is impossible to fulfill with the given schema (e.g., "Which employee made the most sales?"), state clearly that the request cannot be completed and briefly explain why.

-----

### **EXAMPLES**

**Example 1: Simple Lookup**

  * **User Query:** "Show me all customers who live in Noida"
  * **Expected Output:**
    ```json
    {
      "status": "success",
      "response": "SELECT * FROM customers WHERE city = 'Noida';"
    }
    ```

**Example 2: Complex Join and Aggregation**

  * **User Query:** "What are the names of the top 3 products with the highest total revenue?"
  * **Expected Output:**
    ```json
    {
      "status": "success",
      "response": "SELECT T2.product_name FROM orders AS T1 INNER JOIN products AS T2 ON T1.product_id = T2.product_id GROUP BY T2.product_name ORDER BY SUM(T1.total_price) DESC LIMIT 3;"
    }
    ```

**Example 3: Ambiguous Query**

  * **User Query:** "Show me recent orders"
  * **Expected Output:**
    ```json
    {
      "status": "clarification_needed",
      "response": "Could you please define what 'recent' means? For example, 'in the last 7 days', 'this month', or 'since August 2025'."
    }
    ```

**Example 4: Impossible Query**

  * **User Query:** "Which warehouse has the most stock?"
  * **Expected Output:**
    ```json
    {
      "status": "error",
      "response": "I cannot answer this question as the database does not contain information about warehouses."
    }
    ```

-----

### **OUTPUT FORMAT**

Your final response must be a single JSON object with two keys:

1.  `"status"`: A string with one of three possible values: `"success"`, `"clarification_needed"`, or `"error"`.
2.  `"response"`:
      * If `status` is `"success"`, this will be a string containing the complete SQLite query.
      * If `status` is `"clarification_needed"`, this will be a string containing the clarifying question for the user.
      * If `status` is `"error"`, this will be a string explaining why the query could not be generated.
"""

## Add Helper functions

In [14]:
import json
def get_sql_query(genai_client, prompt, user_query):

  # https://www.geeksforgeeks.org/python/formatted-string-literals-f-strings-python/
  contents = f"""
  {prompt}

  Here's the user query in english you need to work on:
  {user_query}
  """
  response = genai_client.models.generate_content(model='gemini-2.5-flash', contents=contents)
  # print(response)

  # Access the usage_metadata attribute
  usage_metadata = response.usage_metadata

  # Print the different token counts
  print(f"Input Token Count: {usage_metadata.prompt_token_count}")
  print(f"Thoughts Token Count: {response.usage_metadata.thoughts_token_count}")
  print(f"Output Token Count: {usage_metadata.candidates_token_count}")
  print(f"Total Token Count: {usage_metadata.total_token_count}")

  output = json.loads(response.text.replace('```json', '').replace('```', ''))

  return output

In [15]:
import sqlite3
import pandas as pd

def execute_query(query, db_name='ecommerce.db'):

    conn = None
    try:
        # Connect to the database
        conn = sqlite3.connect(db_name)
        cursor = conn.cursor()

        # Execute the query
        print(f"\nExecuting query on '{db_name}':\n{query}")
        cursor.execute(query)

        # Fetch all results
        results = cursor.fetchall()

        # Get column names from the cursor description
        columns = [description[0] for description in cursor.description]

        # Format results as a dataframe for easier use
        results_as_dict = [dict(zip(columns, row)) for row in results]
        results_df = pd.DataFrame(results_as_dict)

        print("Query executed successfully.")
        return results_df

    except sqlite3.Error as e:
        print(f"Database error executing query: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None
    finally:
        if conn:
            conn.close()

Testing with SQL queries

In [16]:
def text2sql(genai_client, prompt, user_query):
  output = get_sql_query(genai_client, prompt, user_query)
  if output['status'] == 'success':
    results = execute_query(output['response'])
    return results
  return output

In [17]:
text2sql(genai_client, prompt, "Show me the names and email adress of employees hired after 2020")

Input Token Count: 1645
Thoughts Token Count: 159
Output Token Count: 52
Total Token Count: 1856

Executing query on 'ecommerce.db':
SELECT first_name, last_name, email FROM employees WHERE STRFTIME('%Y', hire_date) > '2020';
Query executed successfully.


,first_name,last_name,email
0,Ferris,Greenway,fgreenway9@technorati.com
1,North,O'Lenechan,nolenechanu@toplist.cz
2,Harwilll,Tidridge,htidridge13@hud.gov
3,Arabele,Beglin,abeglin19@smugmug.com
4,Marnie,Peteri,mpeteri1g@geocities.jp
...,...,...,...
66,Brade,Calbreath,bcalbreathpy@usatoday.com
67,Lila,Leat,lleatq4@ebay.com
68,Earvin,Milesop,emilesopqb@elpais.com
69,Ode,Portail,oportailqk@nytimes.com


In [18]:
text2sql(genai_client, prompt, "Which employee made the most sales?")

Input Token Count: 1636
Thoughts Token Count: 299
Output Token Count: 40
Total Token Count: 1975


{'status': 'error',
 'response': 'I cannot answer this question as the database does not link employees to specific sales or orders.'}

In [19]:
text2sql(genai_client, prompt, "List employees earning more than $10,000")

Input Token Count: 1641
Thoughts Token Count: 88
Output Token Count: 36
Total Token Count: 1765

Executing query on 'ecommerce.db':
SELECT * FROM employees WHERE salary > 10000;
Query executed successfully.


,employee_id,first_name,last_name,email,gender,department,job_title,salary,hire_date
0,1,Blinni,Salaman,bsalaman0@biblegateway.com,Female,HR,Environmental Tech,76600.20,2018-02-25 00:00:00
1,2,Ainslee,Balcers,abalcers1@cpanel.net,Female,Marketing,Research Assistant IV,33484.14,2013-09-02 00:00:00
2,3,Thomasine,Liebrecht,tliebrecht2@tamu.edu,Bigender,Marketing,Senior Developer,36691.01,2017-07-25 00:00:00
3,4,Erasmus,Long,elong3@yale.edu,Male,Marketing,Systems Administrator II,45727.19,2012-03-16 00:00:00
4,5,Farrell,Timmons,ftimmons4@ibm.com,Male,HR,Structural Engineer,75516.15,2010-09-29 00:00:00
...,...,...,...,...,...,...,...,...,...
995,996,Frannie,Essery,fesseryrn@cnbc.com,Male,HR,Registered Nurse,35281.07,2020-10-01 00:00:00
996,997,Brok,Rogger,broggerro@adobe.com,Male,Sales,Dental Hygienist,92971.96,2011-03-24 00:00:00
997,998,Jerrie,Chatell,jchatellrp@chicagotribune.com,Bigender,HR,Structural Analysis Engineer,67002.22,2015-06-02 00:00:00
998,999,Elden,Larcier,elarcierrq@bluehost.com,Polygender,Sales,Marketing Assistant,75909.34,2015-05-29 00:00:00


## **Conclusion**
- This project successfully demonstrated how large language models, specifically Google’s Gemini API, can bridge the gap between natural language and structured query generation. By integrating Gemini with a mock employee dataset, we explored the model’s ability to interpret human-like prompts and translate them into accurate SQL queries.

- Through iterative testing, we validated Gemini’s strengths in handling:

- Basic data retrieval and filtering

- Aggregations and groupings

- Semantic understanding of vague or complex queries

The use of Google AI Studio for API key management and Colab for execution provided a seamless development environment. Installing the google-generativeai SDK enabled direct interaction with Gemini, allowing for rapid prototyping and evaluation.

Overall, this project highlights the potential of GenAI tools in democratizing data access—making SQL querying more intuitive for non-technical users while offering powerful augmentation for data professionals. Future work could involve fine-tuning prompt strategies, expanding schema coverage, and benchmarking Gemini against other LLMs for text-to-SQL accuracy and efficiency.